In [1]:
#Imports

#Pandas: Reading and analyzing data
import pandas as pd
#Numerical calcuations
import numpy as np
#statistical data visualization
import seaborn as sns
#Use Dates in Datetime Format
import datetime
#Tensorflow
import tensorflow as tf
#Keras: Open-Source deep-learning library 
from tensorflow import keras
#Building blocks of NN in Keras
from tensorflow.keras import layers
#EarlyStop to stop training early
from tensorflow.keras.callbacks import EarlyStopping
#Functional API: Layers for different models
from keras.layers import Dense, LSTM, Dropout
#Normalization
from sklearn.preprocessing import MinMaxScaler
#Standardization
from sklearn.preprocessing import StandardScaler
#Evaluate models
import math
#Evaluate MSE
from sklearn.metrics import mean_squared_error
#plot numpy array
import matplotlib.pyplot as plt
#Create Folder for modelling checkpoint
import os
#Callback to logg model fitting time
import time
from timeit import default_timer as timer
#Clear output after training
import IPython
import IPython.display
#Normalization
from sklearn.preprocessing import MinMaxScaler
#Standardization
from sklearn.preprocessing import StandardScaler

#Helper Class (Export Notebook as .py)
# helper functions
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
cwd = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
sys.path.insert(1, cwd + "/src/d00_utils") 
from local_helper_functions import *
from model_helper_functions import *
from windowgenerator import *

#Data Analytics

print("Get data")
#Read CSV file to pandas dataframe; encoding= 'unicode_escape': Decode from Latin-1 source code. Default UTF-8.
df = pd.read_csv(cwd+'/data/d03_data_processed/d03_data_processed.csv', encoding= 'unicode_escape', index_col='Date')
#Display smart meter names and amount
smart_meter_names = df.columns[2:-4]
print("Selected clients: ", len(smart_meter_names))

# Make Datasets for the 33 clients and for 5 and 7 features
ds_dict = makeDatasetsForclientsAndfeatures(smart_meter_names, df)
print("Created dictionary with datasets")

#Set Hyperparameter
#Data Shape
OUT_STEPS = [12, 24] #Next 12 or 24 hours
NUM_FEATURES = [5, 7] # [F_T, F_TW] load_value, hour sin, hour cos, dayofweek sin, dayofweek cos + (temp, rhum)
INPUT_STEPS = 24
INPUT_SHAPE = [(INPUT_STEPS, NUM_FEATURES[0]), (INPUT_STEPS, NUM_FEATURES[1])]

#LSTM
NUM_LSTM_LAYERS = 4
NUM_LSTM_CELLS = 32
NUM_LSTM_DENSE_LAYERS=1
NUM_LSTM_DENSE_UNITS = 32
LSTM_DROPOUT = 0.2

#CNN
CONV_WIDTH = 3
NUM_CNN_LAYERS = 4
NUM_CNN_FILTERS = 24
NUM_CNN_DENSE_LAYERS = 1
NUM_CNN_DENSE_UNITS = 32
CNN_DROPOUT = 0.2

#Federated Learning
comms_round = 20
#Training epochs
MAX_EPOCHS = 100

# Create Windows 
windows_dict = createLocalDataWindows(smart_meter_names, INPUT_STEPS, OUT_STEPS, ds_dict)
print("Created Data windows")

# Local Learning
# Set random seed for as reproducible results as possible
tf.random.set_seed(42)

#h12 f5

#Build and save local models
for idx, client in enumerate(smart_meter_names):
    #Build local models (LSTM, CNN, Transformer)
    local_LSTM_models, local_CNN_models, local_Transformer_models = createLocalModels(
        smart_meter_names, INPUT_SHAPE[0], OUT_STEPS[0], NUM_FEATURES[0], 'Local_LSTM_F5_H12', 'Local_CNN_F5_H12', 'Local_Transformer_F5_H12',
        NUM_LSTM_CELLS, NUM_LSTM_LAYERS, NUM_LSTM_DENSE_LAYERS, NUM_LSTM_DENSE_UNITS, LSTM_DROPOUT, 
        CONV_WIDTH, NUM_CNN_LAYERS, NUM_CNN_FILTERS, NUM_CNN_DENSE_LAYERS, NUM_CNN_DENSE_UNITS, CNN_DROPOUT, 
    )
    #initilally save local models
    initiallySaveAllLocalModels(cwd, smart_meter_names, local_LSTM_models, local_CNN_models, local_Transformer_models)
  


#Train local models
for idx, client in enumerate(smart_meter_names): 
    
    # Clear terminal and print current training round
    IPython.display.clear_output()
    print("Started with new client -----------------", idx+1, "/33")
    
    #Get local models
    local_LSTM_model, local_CNN_model, local_Transformer_model = loadLocalModels( 
        cwd, local_LSTM_models, local_CNN_models, local_Transformer_models, idx, client
    )
    
    #compile and fit for n rounds
    local_LSTM_model = compile_fit(
        local_LSTM_model,
        windows_dict[client][0],
        MAX_EPOCHS
    )
       
    #compile and fit n rounds
    local_CNN_model = compile_fit(
        local_LSTM_model,
        windows_dict[client][0],
        MAX_EPOCHS
    )
    
    #Compile and fit n rounds
    local_Transformer_model = compile_fit(
        local_LSTM_model,
        windows_dict[client][0],
        MAX_EPOCHS
    )
        
    #Save Transformer model
    saveLocalModels(cwd, local_LSTM_model, local_CNN_model, local_Transformer_model, client)

In [81]:
#Build Models
multi_lstm_model = build_LSTM_model(
    input_shape = INPUT_SHAPE[0], 
    num_LSTM_cells = NUM_LSTM_CELLS,
    num_LSTM_layers = NUM_LSTM_LAYERS,
    num_LSTM_dense_layers = NUM_LSTM_DENSE_LAYERS,
    num_LSTM_dense_units = NUM_LSTM_DENSE_UNITS,
    LSTM_dropout = LSTM_DROPOUT,
    output_steps = OUT_STEPS[1],
    num_features = NUM_FEATURES[0],
    model_name = 'Local_LSTM_F5_H24'
)
multi_conv_model = build_CNN_model(
    input_shape = INPUT_SHAPE[0], 
    conv_width = CONV_WIDTH,
    num_CNN_layers = NUM_CNN_LAYERS,
    num_CNN_filters = NUM_CNN_FILTERS,
    num_CNN_dense_layers = NUM_CNN_DENSE_LAYERS,
    num_CNN_dense_units = NUM_CNN_DENSE_UNITS,
    CNN_dropout = CNN_DROPOUT,
    output_steps = OUT_STEPS[1],
    num_features = NUM_FEATURES[0],
    model_name = 'Local_CNN_F5_H24'
)
#Transformer
multi_transformer_model = build_transformer_model(
    input_shape = INPUT_SHAPE[0],
    head_size=TRANSFORMER_HEAD_SIZE,
    num_heads=TRANSFORMER_NUM_HEADS,
    ff_dim=TRANSFORMER_FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[TRANSFORMER_MLP_UNITS],
    mlp_dropout=TRANSFORMER_MLP_DROPOUT,
    dropout=TRANSFORMER_DROPOUT,
    out_steps = OUT_STEPS[1],
    num_features = NUM_FEATURES[0],
    model_name = 'Local_Transformer_F5_H24'    
)

In [82]:
#windows_dict[smart_meter_names][0-3] 
#    -> 0:window_F5_H12 , 1:window_F5_H24 , 2:window_F7_H12 , 3:window_F7_H24
forecasts_dict_LSTM_F5_H24 = {}
forecasts_dict_CNN_F5_H24 = {}
forecasts_dict_Transformer_F5_H24 = {}

for idx, client in enumerate(smart_meter_names):
    IPython.display.clear_output()
    print("-----------------------", idx, "/32")
    
    #LSTM
    history = compile_and_fit(multi_lstm_model, windows_dict[client][1], model_name='Local_LSTM_F5_H24', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_LSTM_F5_H24/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][1].test)
    #Save results
    forecasts_dict_LSTM_F5_H24[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
    #CNN
    history = compile_and_fit(multi_conv_model, windows_dict[client][1], model_name='Local_CNN_F5_H24', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_CNN_F5_H24/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][1].test)
    #Save results
    forecasts_dict_CNN_F5_H24[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
    #Transformer
    history = compile_and_fit(multi_transformer_model, windows_dict[client][1], model_name='Local_Transformer_F5_H24', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_Transformer_F5_H24/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][1].test)
    #Save results
    forecasts_dict_Transformer_F5_H24[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
final_dict['Local']['LSTM']['H24']['F5'] = forecasts_dict_LSTM_F5_H24
final_dict['Local']['CNN']['H24']['F5'] = forecasts_dict_CNN_F5_H24
final_dict['Local']['Transformer']['H24']['F5'] = forecasts_dict_Transformer_F5_H24

----------------------- 32 /32
574/574 [==============================] - ETA: 0s - loss: 0.0372 - root_mean_squared_error: 0.1929 - mean_absolute_percentage_error: 40866524.0000 - mean_absolute_error: 0.1440

INFO:tensorflow:Assets written to: model_experiments/Local/Local_LSTM_F5_H24\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_LSTM_F5_H24\6907-ZE01-74\assets


565/574 [============================>.] - ETA: 0s - loss: 0.0386 - root_mean_squared_error: 0.1964 - mean_absolute_percentage_error: 40684024.0000 - mean_absolute_error: 0.1468

INFO:tensorflow:Assets written to: model_experiments/Local/Local_CNN_F5_H24\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_CNN_F5_H24\6907-ZE01-74\assets


574/574 [==============================] - ETA: 0s - loss: 0.0365 - root_mean_squared_error: 0.1910 - mean_absolute_percentage_error: 41127916.0000 - mean_absolute_error: 0.1416

INFO:tensorflow:Assets written to: model_experiments/Local/Local_Transformer_F5_H24\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_Transformer_F5_H24\6907-ZE01-74\assets


81/81 [==============================] - 2s 18ms/step - loss: 0.0372 - root_mean_squared_error: 0.1929 - mean_absolute_percentage_error: 37018404.0000 - mean_absolute_error: 0.1606


### Features 7, Horizon 12

In [83]:
#Build Models
multi_lstm_model = build_LSTM_model(
    input_shape = INPUT_SHAPE[1], 
    num_LSTM_cells = NUM_LSTM_CELLS,
    num_LSTM_layers = NUM_LSTM_LAYERS,
    num_LSTM_dense_layers = NUM_LSTM_DENSE_LAYERS,
    num_LSTM_dense_units = NUM_LSTM_DENSE_UNITS,
    LSTM_dropout = LSTM_DROPOUT,
    output_steps = OUT_STEPS[0],
    num_features = NUM_FEATURES[1],
    model_name = 'Local_LSTM_F7_H12'
)
multi_conv_model = build_CNN_model(
    input_shape = INPUT_SHAPE[1], 
    conv_width = CONV_WIDTH,
    num_CNN_layers = NUM_CNN_LAYERS,
    num_CNN_filters = NUM_CNN_FILTERS,
    num_CNN_dense_layers = NUM_CNN_DENSE_LAYERS,
    num_CNN_dense_units = NUM_CNN_DENSE_UNITS,
    CNN_dropout = CNN_DROPOUT,
    output_steps = OUT_STEPS[0],
    num_features = NUM_FEATURES[1],
    model_name = 'Local_CNN_F7_H12'
)
#Transformer
multi_transformer_model = build_transformer_model(
    input_shape = INPUT_SHAPE[1],
    head_size=TRANSFORMER_HEAD_SIZE,
    num_heads=TRANSFORMER_NUM_HEADS,
    ff_dim=TRANSFORMER_FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[TRANSFORMER_MLP_UNITS],
    mlp_dropout=TRANSFORMER_MLP_DROPOUT,
    dropout=TRANSFORMER_DROPOUT,
    out_steps = OUT_STEPS[0],
    num_features = NUM_FEATURES[1],
    model_name = 'Local_Transformer_F7_H12'    
)

In [84]:
#windows_dict[smart_meter_names][0-3] 
#    -> 0:window_F5_H12 , 1:window_F5_H24 , 2:window_F7_H12 , 3:window_F7_H24
forecasts_dict_LSTM_F7_H12 = {}
forecasts_dict_CNN_F7_H12 = {}
forecasts_dict_Transformer_F7_H12 = {}

for idx, client in enumerate(smart_meter_names):
    IPython.display.clear_output()
    print("-----------------------", idx, "/32")
    
    #LSTM
    history = compile_and_fit(multi_lstm_model, windows_dict[client][2], model_name='Local_LSTM_F7_H12', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_LSTM_F7_H12/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][2].test)
    #Save results
    forecasts_dict_LSTM_F7_H12[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
    #CNN
    history = compile_and_fit(multi_conv_model, windows_dict[client][2], model_name='Local_CNN_F7_H12', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_CNN_F7_H12/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][2].test)
    #Save results
    forecasts_dict_CNN_F7_H12[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
    #Transformer
    history = compile_and_fit(multi_transformer_model, windows_dict[client][2], model_name='Local_Transformer_F7_H12', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_Transformer_F7_H12/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][2].test)
    #Save results
    forecasts_dict_Transformer_F7_H12[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
final_dict['Local']['LSTM']['H12']['F7'] = forecasts_dict_LSTM_F7_H12
final_dict['Local']['CNN']['H12']['F7'] = forecasts_dict_CNN_F7_H12
final_dict['Local']['Transformer']['H12']['F7'] = forecasts_dict_Transformer_F7_H12

----------------------- 32 /32
575/575 [==============================] - ETA: 0s - loss: 0.0320 - root_mean_squared_error: 0.1789 - mean_absolute_percentage_error: 40454040.0000 - mean_absolute_error: 0.1291

INFO:tensorflow:Assets written to: model_experiments/Local/Local_LSTM_F7_H12\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_LSTM_F7_H12\6907-ZE01-74\assets


570/575 [============================>.] - ETA: 0s - loss: 0.0346 - root_mean_squared_error: 0.1861 - mean_absolute_percentage_error: 40843364.0000 - mean_absolute_error: 0.1362

INFO:tensorflow:Assets written to: model_experiments/Local/Local_CNN_F7_H12\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_CNN_F7_H12\6907-ZE01-74\assets


574/575 [============================>.] - ETA: 0s - loss: 0.0347 - root_mean_squared_error: 0.1864 - mean_absolute_percentage_error: 41131696.0000 - mean_absolute_error: 0.1375

INFO:tensorflow:Assets written to: model_experiments/Local/Local_Transformer_F7_H12\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_Transformer_F7_H12\6907-ZE01-74\assets


82/82 [==============================] - 2s 23ms/step - loss: 0.0369 - root_mean_squared_error: 0.1921 - mean_absolute_percentage_error: 36155700.0000 - mean_absolute_error: 0.1631


### Features 7, Horizon 24

In [85]:
#Build Models
multi_lstm_model = build_LSTM_model(
    input_shape = INPUT_SHAPE[1], 
    num_LSTM_cells = NUM_LSTM_CELLS,
    num_LSTM_layers = NUM_LSTM_LAYERS,
    num_LSTM_dense_layers = NUM_LSTM_DENSE_LAYERS,
    num_LSTM_dense_units = NUM_LSTM_DENSE_UNITS,
    LSTM_dropout = LSTM_DROPOUT,
    output_steps = OUT_STEPS[1],
    num_features = NUM_FEATURES[1],
    model_name = 'Local_LSTM_F7_H24'
)
multi_conv_model = build_CNN_model(
    input_shape = INPUT_SHAPE[1], 
    conv_width = CONV_WIDTH,
    num_CNN_layers = NUM_CNN_LAYERS,
    num_CNN_filters = NUM_CNN_FILTERS,
    num_CNN_dense_layers = NUM_CNN_DENSE_LAYERS,
    num_CNN_dense_units = NUM_CNN_DENSE_UNITS,
    CNN_dropout = CNN_DROPOUT,
    output_steps = OUT_STEPS[1],
    num_features = NUM_FEATURES[1],
    model_name = 'Local_CNN_F7_H24'
)
#Transformer
multi_transformer_model = build_transformer_model(
    input_shape = INPUT_SHAPE[1],
    head_size=TRANSFORMER_HEAD_SIZE,
    num_heads=TRANSFORMER_NUM_HEADS,
    ff_dim=TRANSFORMER_FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[TRANSFORMER_MLP_UNITS],
    mlp_dropout=TRANSFORMER_MLP_DROPOUT,
    dropout=TRANSFORMER_DROPOUT,
    out_steps = OUT_STEPS[1],
    num_features = NUM_FEATURES[1],
    model_name = 'Local_Transformer_F7_H24'    
)

In [86]:
#windows_dict[smart_meter_names][0-3] 
#    -> 0:window_F5_H12 , 1:window_F5_H24 , 2:window_F7_H12 , 3:window_F7_H24
forecasts_dict_LSTM_F7_H24 = {}
forecasts_dict_CNN_F7_H24 = {}
forecasts_dict_Transformer_F7_H24 = {}

for idx, client in enumerate(smart_meter_names):
    IPython.display.clear_output()
    print("-----------------------", idx, "/32")
    
    #LSTM
    history = compile_and_fit(multi_lstm_model, windows_dict[client][3], model_name='Local_LSTM_F7_H24', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_LSTM_F7_H24/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][3].test)
    #Save results
    forecasts_dict_LSTM_F7_H24[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
    #CNN
    history = compile_and_fit(multi_conv_model, windows_dict[client][3], model_name='Local_CNN_F7_H24', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_CNN_F7_H24/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][3].test)
    #Save results
    forecasts_dict_CNN_F7_H24[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
    #Transformer
    history = compile_and_fit(multi_transformer_model, windows_dict[client][3], model_name='Local_Transformer_F7_H24', client_name=client, MAX_EPOCHS=MAX_EPOCHS)
    #Load best model
    model = tf.keras.models.load_model(f"model_experiments/Local/Local_Transformer_F7_H24/{client}")
    model_evaluation_test = model.evaluate(windows_dict[client][3].test)
    #Save results
    forecasts_dict_Transformer_F7_H24[client] = {
        'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
        'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1))                              
                             }
    
final_dict['Local']['LSTM']['H24']['F7'] = forecasts_dict_LSTM_F7_H24
final_dict['Local']['CNN']['H24']['F7'] = forecasts_dict_CNN_F7_H24
final_dict['Local']['Transformer']['H24']['F7'] = forecasts_dict_Transformer_F7_H24

----------------------- 32 /32
574/574 [==============================] - ETA: 0s - loss: 0.0361 - root_mean_squared_error: 0.1901 - mean_absolute_percentage_error: 40855784.0000 - mean_absolute_error: 0.1421

INFO:tensorflow:Assets written to: model_experiments/Local/Local_LSTM_F7_H24\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_LSTM_F7_H24\6907-ZE01-74\assets


570/574 [============================>.] - ETA: 0s - loss: 0.0386 - root_mean_squared_error: 0.1965 - mean_absolute_percentage_error: 40561624.0000 - mean_absolute_error: 0.1483

INFO:tensorflow:Assets written to: model_experiments/Local/Local_CNN_F7_H24\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_CNN_F7_H24\6907-ZE01-74\assets


574/574 [==============================] - ETA: 0s - loss: 0.0369 - root_mean_squared_error: 0.1920 - mean_absolute_percentage_error: 41121532.0000 - mean_absolute_error: 0.1436

INFO:tensorflow:Assets written to: model_experiments/Local/Local_Transformer_F7_H24\6907-ZE01-74\assets


INFO:tensorflow:Assets written to: model_experiments/Local/Local_Transformer_F7_H24\6907-ZE01-74\assets


81/81 [==============================] - 3s 29ms/step - loss: 0.0382 - root_mean_squared_error: 0.1954 - mean_absolute_percentage_error: 35927888.0000 - mean_absolute_error: 0.1646


# Evaluation

In [89]:
final_dict['Local']['LSTM']['H24']['F7']['0101-ZE01-70']

{'MSE': 0.019874952733516693,
 'RMSE': 0.14097857475280762,
 'MAPE': 68598168.0,
 'MAE': 0.11403103917837143,
 'Time': 99.57683070003986}

In [90]:
import pickle 
with open('Dictionaries_Results/Local_results.pkl', 'wb') as f:
    pickle.dump(final_dict, f)

In [91]:
with open('Dictionaries_Results/Local_results.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
#loaded_dict